In [2]:
#Import the necessary libraries 
import requests 
import json 
from env.api import openai_api_key
import openai
from bs4 import BeautifulSoup

# imports
import pandas as pd
import numpy as np

openai.api_key = openai_api_key

In [8]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Regulations_Data/WI/2022WI_HuntRegulations.pdf")
pages = loader.load()

In [18]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

In [19]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()


Using embedded DuckDB without persistence: data will be transient


In [21]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "can I hunt deer with a 9mm pistol as long I use powerful ammunition?"

docs=retriever.get_relevant_documents(query)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)




InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 8195 tokens (7939 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [6]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders([loader])


Using embedded DuckDB without persistence: data will be transient


In [7]:
index.query_with_sources("can I hunt deer with a 9mm pistol as long I use powerful ammunition?")


{'question': 'what types of handguns are legal for hunting deer?',
 'answer': ' Handguns are not legal for hunting deer in Wisconsin.\n',
 'sources': 'Regulations_Data/WI/2022WI_HuntRegulations.pdf'}